# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j

# For maps
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 5.3MB/s 


Define the environment (Java & Spark homes)

---

In [2]:
!ls /content

sample_data  spark-3.0.1-bin-hadoop3.2	spark-3.0.1-bin-hadoop3.2.tgz


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Starting Spark Session and print the version


---


In [4]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4500") \
        .getOrCreate()

spark.version

'3.0.1'

In [5]:
spark

In [6]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [10]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4500 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-11-28 08:12:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.44.17.83, 54.227.160.247, 3.226.231.47, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.44.17.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  39.9MB/s    in 0.3s    

2020-11-28 08:12:26 (39.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://af4632d21ff5.ngrok.io


# Descargar Datasets

In [8]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/frankenstein.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/el_quijote.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!ls /dataset

characters.csv	el_quijote.txt	frankenstein.txt  planets.csv


# RDD

---



## Ejemplo 1

In [16]:
frankenstein = spark.sparkContext.textFile("/dataset/frankenstein.txt")
frankenstein.first()

'FRANKENSTEIN'

Abre la Spark UI e investiga qué ha sucedido


---




## Creación de colecciones paralelizadas
Una manera muy rápida de crear RDD desde la shell, cuando estamos aprendiendo, es crear una colección paralelizada. Para ello:

## Ejemplo 2

In [11]:
distData = spark.sparkContext.parallelize([25, 20, 15, 10, 5])
distData.reduce(lambda x ,y: x + y)

75

¿De qué tipo es la variable distData?


*Spark RDD (Resilient Distributed Dataset)*

## Ejercicio 1
Cuenta el número de líneas del fichero "el_quijote.txt"

---



In [12]:
# Es buena praxis aplicar cada función en una línea
# \ se utiliza para decirle que siga en la siguiente línea
quijote = spark \
.sparkContext \
.textFile("/dataset/el_quijote.txt")

quijote

/dataset/el_quijote.txt MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
quijote.count()

2186

## Ejercicio 2
Imprime la primera línea del fichero "el_quijote.txt"

---



In [14]:
quijote.first()

'DON QUIJOTE DE LA MANCHA'

## Transformaciones y Acciones sobre RDDs 

### Acciones

### Ejemplo 3

In [17]:
print(frankenstein.count()) # Número de elementos en el RDD
print(frankenstein.first()) # Primer elemento del RDD

7237
FRANKENSTEIN


### Transformaciones

### Ejemplo 4

In [18]:
# ReduceByKey
lines = spark.sparkContext.textFile("/dataset/frankenstein.txt")
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b).cache()
counts.count()
counts.collect() # show the RDD

[('FRANKENSTEIN', 1),
 ('', 811),
 ('Letter 1', 1),
 ('commencement of an enterprise which you have regarded with such evil', 1),
 ('forebodings.  I arrived here yesterday, and my first task is to assure', 1),
 ('my dear sister of my welfare and increasing confidence in the success', 1),
 ('of my undertaking.', 1),
 ('feeling?  This breeze, which has travelled from the regions towards', 1),
 ('frost and desolation; it ever presents itself to my imagination as the', 1),
 ('region of beauty and delight.  There, Margaret, the sun is forever', 1),
 ('perpetual splendour.  There—for with your leave, my sister, I will put', 1),
 ('some trust in preceding navigators—there snow and frost are banished;', 1),
 ('solitudes.  What may not be expected in a country of eternal light?  I', 1),
 ('shall satiate my ardent curiosity with the sight of a part of the world',
  1),
 ('never before visited, and may tread a land never before imprinted by', 1),
 ('the foot of man. These are my enticements, and 

In [19]:
# SortByKey
sorted = counts.sortByKey()
sorted.collect()

[('', 811),
 ('                                                "Elizabeth Lavenza', 1),
 ('                                                August 26th, 17—', 1),
 ('                              "Alphonse Frankenstein.', 1),
 ('                    [Wordsworth\'s "Tintern Abbey".]', 1),
 ('               "Your affectionate and afflicted father,', 1),
 ('               Walton, in continuation.', 1),
 ('     An appetite; a feeling, and a love,', 1),
 ('     By thought supplied, or any interest', 1),
 ('     Haunted him like a passion:  the tall rock,', 1),
 ('     That had no need of a remoter charm,', 1),
 ('     The mountain, and the deep and gloomy wood,', 1),
 ('     Their colours and their forms, were then to him', 1),
 ("     Unborrow'd from the eye.", 1),
 ('     ——The sounding cataract', 1),
 ('    "Geneva, May 18th, 17—"', 1),
 ('   Embrace fond woe, or cast our cares away;', 1),
 ('   Nought may endure but mutability!', 1),
 ('   The path of its departure still is free.', 1),
 (

Abre la Spark UI e investiga que ha ido pasando.

### Ejemplo 5

In [20]:
# Filter
linesWithSpark = textFile.filter(lambda line: "the" in line)
linesWithSpark.count()

3712

### Ejercicio 3
Obtén el número de ocurrencias de cada palabra del fichero "frankenstein.txt"

In [ ]:
# ReduceByKey
lines = frankenstein
word_pairs = lines.flatMap(lambda line: line.split(' ')) \
.map(lambda s: (s, 1))

counts = word_pairs.reduceByKey(lambda a, b: a + b) \
.cache()

counts.collect() # show the RDD

### Ejercicio 4
Obtén el top-10 de palabras con mayor recurrencia

---



In [48]:
# Change the Value to become the Key and sort by the Value
changed = counts.map(lambda s: (s[1], s[0])) \
.sortByKey(ascending=True)

In [49]:
# Show top 10
changed.top(10)

[(3897, 'the'),
 (3488, ''),
 (2904, 'and'),
 (2720, 'I'),
 (2634, 'of'),
 (2072, 'to'),
 (1629, 'my'),
 (1338, 'a'),
 (1072, 'in'),
 (992, 'was')]

In [50]:
# Take the first X num of items whatever they are (even if they are not ordered)
# To show the same result as top(10), change ascending=False in sortByKey
changed.take(10)

[(1, 'FRANKENSTEIN'),
 (1, 'Mary'),
 (1, 'Wollstonecraft'),
 (1, '(Godwin)'),
 (1, 'Shelley'),
 (1, 'enterprise'),
 (1, 'forebodings.'),
 (1, 'yesterday,'),
 (1, 'braces'),
 (1, 'feeling?')]

### Ejercicio 5
Obtén el top 10 de paralabras con más de 4 caracteres

In [54]:
words = frankenstein
words.flatMap(lambda line: line.split(' ')) \
.filter(lambda w: len(w) > 4) \
.map(lambda s: (s, 1)) \
.reduceByKey(lambda a, b: a + b) \
.map(lambda s: (s[1], s[0])) \
.top(10)
# OR
# .reduceByKey().take(10)
# Si queremos obtener el last 10 -> .reduceByKey(False).take()

[(540, 'which'),
 (187, 'could'),
 (177, 'would'),
 (174, 'their'),
 (152, 'should'),
 (130, 'these'),
 (122, 'before'),
 (107, 'might'),
 (105, 'myself'),
 (103, 'every')]

## Key/Value Pair RDD

---



### Ejemplo 6


---



In [55]:
charac_sw = spark.sparkContext.textFile("/dataset/characters.csv")
planets_sw = spark.sparkContext.textFile("/dataset/planets.csv")
charac_sw.take(10)

['name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species',
 'Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,Human',
 'C-3PO,167,75,NA,gold,yellow,112BBY,NA,Tatooine,Droid',
 'R2-D2,96,32,NA,"white, blue",red,33BBY,NA,Naboo,Droid',
 'Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,Human',
 'Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,Human',
 'Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,Tatooine,Human',
 'Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,Tatooine,Human',
 'R5-D4,97,32,NA,"white, red",red,NA,NA,Tatooine,Droid',
 'Biggs Darklighter,183,84,black,light,brown,24BBY,male,Tatooine,Human']

In [56]:
planets_sw.take(10)

['name;rotation_period;orbital_period;diameter;climate;gravity;terrain;surface_water;population',
 'Alderaan;24;364;12500;temperate;1 standard;grasslands, mountains;40;2000000000',
 'Yavin IV;24;4818;10200;temperate, tropical;1 standard;jungle, rainforests;8;1000',
 'Hoth;23;549;7200;frozen;1.1 standard;tundra, ice caves, mountain ranges;100;NA',
 'Dagobah;23;341;8900;murky;N/A;swamp, jungles;8;NA',
 'Bespin;12;5110;118000;temperate;1.5 (surface), 1 standard (Cloud City);gas giant;0;6000000',
 'Endor;18;402;4900;temperate;0.85 standard;forests, mountains, lakes;8;30000000',
 'Naboo;26;312;12120;temperate;1 standard;grassy hills, swamps, forests, mountains;12;4500000000',
 'Coruscant;24;368;12240;temperate;1 standard;cityscape, mountains;NA;1E+12',
 'Kamino;27;463;19720;temperate;1 standard;ocean;100;1000000000']

In [57]:
from itertools import islice

charac_sw_noheader = charac_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

planets_sw_noheader = planets_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

### Ejercicio 6
Obtén un listado con la población del planeta al que pertenece cada personaje de Star Wars


---


In [76]:
# First, we form a key-value tuple with the name of the planet as key for both RDD
planets_kv = planets_sw_noheader \
.map(lambda l: l.split(";")) \
.map(lambda l: (l[0], l[-1]))

charac_kv = charac_sw_noheader \
.map(lambda l: l.split(",")) \
.map(lambda l: (l[-2], l[0]))

# Then, we join both datasets and beautify the result (distinct -> remove duplicates)
planets_kv.join(charac_kv) \
.map(lambda x: (x[1][1], x[0], x[1][0])) \
.distinct() \
.take(10)

[('Tarfful', 'Kashyyyk', '45000000'),
 ('Darth Maul', 'Dathomir', '5200'),
 ('Roos Tarpals', 'Naboo', '4500000000'),
 ('Rugor Nass', 'Naboo', '4500000000'),
 ('Ric Olié', 'Naboo', '4500000000'),
 ('Padmé Amidala', 'Naboo', '4500000000'),
 ('Watto', 'Toydaria', '11000000'),
 ('Barriss Offee', 'Mirial', 'NA'),
 ('Sly Moore', 'Umbara', 'NA'),
 ('Lama Su', 'Kamino', '1000000000')]